In [1]:
import ipyrad as ip
print ip.__version__

DEBUG:ipyrad:H4CKERZ-mode: __loglevel__ = DEBUG


0.1.32


In [2]:
data1 = ip.Assembly("Ron")


  New Assembly: Ron


In [3]:
data1.set_params("working_directory", "Ron")
data1.set_params("sorted_fastq_path", "/home/deren/Downloads/Ron/*.gz")
data1.set_params("datatype", "pairddrad")
data1.set_params("restriction_overhang", ("TGCAG", ""))
data1.set_params("max_low_qual_bases", 6)
data1.set_params("filter_adapters", 2)
data1.set_params("max_Hs_consens", (4, 4))
data1.set_params("max_Ns_consens", (4, 4))
data1.set_params("max_Indels_locus", (6, 6))
data1.set_params("edit_cutsites", (6, 4))
data1.set_params("filter_min_trim_len", 50)

data1.get_params()

  1   working_directory           ./Ron                                        
  2   raw_fastq_path              ./*.fastq                                    
  3   barcodes_path               ./*.barcodes.txt                             
  4   sorted_fastq_path           /home/deren/Downloads/Ron/*.gz               
  5   assembly_method             denovo                                       
  6   reference_sequence                                                       
  7   datatype                    pairddrad                                    
  8   restriction_overhang        ('TGCAG', '')                                
  9   max_low_qual_bases          6                                            
  10  phred_Qscore_offset         33                                           
  11  mindepth_statistical        6                                            
  12  mindepth_majrule            6                                            
  13  maxdepth                    1000  

In [4]:
## smaller preview
data1._hackersonly["preview_truncate_length"] = 50000

In [5]:
data1.link_fastqs(force=True)
print data1.stats

## it's nice to have a save function, maybe this shouldn't be hidden.
#data1.save()

4 new Samples created in `Ron`.
4 fastq files linked to 4 new Samples.
                                   state  reads_raw
12_RE-05-11-4_CU_AATGGC-CGCGTCTA       1    3981908
34_RC-06-3-32_STX_AATGGC-TTGCACCC      1    4942688
37_RE-05-1-10_SXM_AATGGC-TACGTAAA      1    3730551
40_19_AATGGC-TCTACCGT                  1    5218462


### Get barcodes from sample names
We'll also shorten the sample names so they read a little prettier. According to the documents Ron sent docs showing that the I7 barcode will be at the end of read1 if it overshoots the fragment length. 

In [6]:
oldnames = data1.samples.keys()

for key in oldnames:
    ## get sample with this name
    sample = data1.samples.pop(key)
    
    ## get new name and barcode from oldname
    newname = sample.name.rsplit("_", 1)[0]
    barcode = sample.name.rsplit('-', 1)[-1]
    
    ## save name and barcode to sample
    sample.name = newname
    sample.barcode = barcode
    
    ## save back to Assembly
    data1.samples[sample.name] = sample
    
    ## check
    print "{}\t{}".format(sample.name, sample.barcode)

34_RC-06-3-32_STX	TTGCACCC
40_19	TCTACCGT
12_RE-05-11-4_CU	CGCGTCTA
37_RE-05-1-10_SXM	TACGTAAA


### Build a barcodes dict since no barcodes file

In [7]:
barcodes = {}
for key, sample in data1.samples.items():
    barcodes[sample.name] = sample.barcode
    
data1.barcodes = barcodes
data1.barcodes

{'12_RE-05-11-4_CU': 'CGCGTCTA',
 '34_RC-06-3-32_STX': 'TTGCACCC',
 '37_RE-05-1-10_SXM': 'TACGTAAA',
 '40_19': 'TCTACCGT'}

In [11]:
import ipyrad as ip
data1 = ip.load.load_assembly("Ron/Ron.assembly")

  loading Assembly: Ron [Ron/Ron.assembly]


### Filter reads

In [8]:
data1.step2(preview=True)
print data1.stats

                   state  reads_raw  reads_filtered
12_RE-05-11-4_CU       2    3981908           30894
34_RC-06-3-32_STX      2    4942688           31877
37_RE-05-1-10_SXM      2    3730551           32075
40_19                  2    5218462           32140


In [11]:
print data1.statsfiles.s2

                   reads_raw  filtered_by_qscore  filtered_by_adapter  \
12_RE-05-11-4_CU       50000               18642                  464   
34_RC-06-3-32_STX      50000               17660                  463   
37_RE-05-1-10_SXM      50000               17456                  469   
40_19                  50000               17375                  485   

                   reads_passed  
12_RE-05-11-4_CU          30894  
34_RC-06-3-32_STX         31877  
37_RE-05-1-10_SXM         32075  
40_19                     32140  


### Problem with high indels filter for paired data still 

In [12]:
data1.step3()

In [13]:
print data1.stats

                   state  reads_raw  reads_filtered  clusters_total  \
12_RE-05-11-4_CU       3    3981908           30894           21744   
34_RC-06-3-32_STX      3    4942688           31877           21300   
37_RE-05-1-10_SXM      3    3730551           32075           21802   
40_19                  3    5218462           32140           21246   

                   clusters_hidepth  
12_RE-05-11-4_CU                100  
34_RC-06-3-32_STX               124  
37_RE-05-1-10_SXM               117  
40_19                           129  


In [14]:
data1.step4()
print data1.stats

                   state  reads_raw  reads_filtered  clusters_total  \
12_RE-05-11-4_CU       4    3981908           30894           21744   
34_RC-06-3-32_STX      4    4942688           31877           21300   
37_RE-05-1-10_SXM      4    3730551           32075           21802   
40_19                  4    5218462           32140           21246   

                   clusters_hidepth  hetero_est  error_est  
12_RE-05-11-4_CU                100    0.024199   0.008354  
34_RC-06-3-32_STX               124    0.015910   0.006538  
37_RE-05-1-10_SXM               117    0.016823   0.007831  
40_19                           129    0.013917   0.006393  


In [1]:
import ipyrad as ip
data1 = ip.load.load_assembly("Ron/Ron.assembly")

data1.step5()
data1.stats

DEBUG:ipyrad:H4CKERZ-mode: __loglevel__ = DEBUG
INFO:ipyrad.core.assembly:try 10: starting controller
DEBUG:ipyrad.core.assembly:OK! Connected to (4) engines


  loading Assembly: Ron [Ron/Ron.assembly]
  Step5: Consensus base calling 

    Skipping: All 4 selected Samples already consensus called


,state,reads_raw,reads_filtered,clusters_total,clusters_hidepth,hetero_est,error_est,reads_consens
12_RE-05-11-4_CU,6,3981908,30894,21744,100,0.024199,0.008354,76
34_RC-06-3-32_STX,6,4942688,31877,21300,124,0.015910,0.006538,107
37_RE-05-1-10_SXM,6,3730551,32075,21802,117,0.016823,0.007831,97
40_19,6,5218462,32140,21246,129,0.013917,0.006393,107


In [3]:
data1.write_params()

In [2]:
data1.step6(force=True)

INFO:ipyrad.core.assembly:try 10: starting controller
DEBUG:ipyrad.core.assembly:OK! Connected to (4) engines
INFO:ipyrad.assemble.cluster_across:creating input files
INFO:ipyrad.assemble.cluster_across:clustering
DEBUG:ipyrad.assemble.cluster_across:/home/deren/Documents/ipyrad/bin/vsearch-linux-x86_64 -cluster_smallmem /home/deren/Documents/ipyrad/tests/Ron/Ron_consens/Ron_cathaps.tmp -leftjust  -query_cov .90  -id 0.85 -userout /home/deren/Documents/ipyrad/tests/Ron/Ron_consens/Ron.utemp -notmatched /home/deren/Documents/ipyrad/tests/Ron/Ron_consens/Ron.htemp -userfields query+target+qstrand -maxaccepts 1 -maxrejects 0 -minsl 0.5 -fasta_width 0 -threads 0 -fulldp  -usersort  -quiet
INFO:ipyrad.assemble.cluster_across:building consens clusters
INFO:ipyrad.assemble.cluster_across:muscle alignment & building indel database
INFO:ipyrad.assemble.cluster_across:building full database
DEBUG:ipyrad.assemble.cluster_across:singlecat: 12_RE-05-11-4_CU
DEBUG:ipyrad.assemble.cluster_across:inse

  Step6: Clustering across 4 samples at 0.85 similarity


IPyradError: Uncaught Exception: shape mismatch: value array of shape (300,4) could not be broadcast to indexing result of shape (301,4)